# IMDb Knowledge Graph Builder for TransE Training

This notebook builds a Knowledge Graph from IMDb TSV files stored in Google Drive.

**Modes:**
- **Full KG**: Processes all movies (produces ~81M triples, ~16.5M entities)
- **Filtered KG**: Filters movies by year, type, and top billing (target: 5-10M triples)

**Entities:**
- Movies (tconst)
- Persons (nconst)
- Genres (from title.basics genres column)

**Relations:**
- (movie) --HAS_GENRE--> (genre)
- (person) --DIRECTED--> (movie)
- (person) --WROTE--> (movie)
- (person) --ACTED_IN--> (movie)

In [ ]:
# Install required packages (if not already installed)
!pip install pandas tqdm -q

In [1]:
# Configuration
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import os
import random

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# ============================================================================
# PIPELINE MODE SELECTION
# ============================================================================
USE_FILTERED_MODE = True  # Set to False for full KG, True for filtered KG

# ============================================================================
# FILTERING CONFIGURATION (only used if USE_FILTERED_MODE = True)
# ============================================================================
YEAR_MIN = 1970  # Keep only movies with startYear >= YEAR_MIN
TOP_BILLING = 5  # Keep only actors/actresses with ordering <= TOP_BILLING

# ============================================================================
# DRY-RUN MODE (for quick testing)
# ============================================================================
LIMIT_ROWS = None  # Set to integer (e.g., 10000) to limit rows per file for testing
LIMIT_MOVIES = None  # Alternative: limit number of movies to process (after filtering)

# ============================================================================
# FULL KG STATISTICS (for comparison - update if you know the full KG stats)
# ============================================================================
FULL_KG_STATS = {
    'movies': 16500000,  # Approximate - update with actual if known
    'triples': 81000000,  # Approximate - update with actual if known
    'entities': 16500000  # Approximate - update with actual if known
}

# ============================================================================
# PATHS AND FILES
# ============================================================================
DRIVE_ROOT = '/content/drive/MyDrive'
INPUT_DIR = f'{DRIVE_ROOT}/Knowledge Graph'  # Folder containing TSV files

# Output directory based on mode
if USE_FILTERED_MODE:
    OUTPUT_DIR = f'{DRIVE_ROOT}/kg_output_filtered'
else:
    OUTPUT_DIR = f'{DRIVE_ROOT}/kg_output'

# File names
FILES = {
    'basics': 'title.basics.tsv',
    'crew': 'title.crew.tsv',
    'principals': 'title.principals.tsv',
    'ratings': 'title.ratings.tsv'  # Not used for triples but can be read
}

print("=" * 60)
print("CONFIGURATION")
print("=" * 60)
print(f"Mode: {'FILTERED KG' if USE_FILTERED_MODE else 'FULL KG'}")
print(f"Input directory: {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
if USE_FILTERED_MODE:
    print(f"  Year minimum: {YEAR_MIN}")
    print(f"  Top billing limit: {TOP_BILLING}")
print(f"Dry-run limit rows: {LIMIT_ROWS if LIMIT_ROWS else 'None'}")
print(f"Dry-run limit movies: {LIMIT_MOVIES if LIMIT_MOVIES else 'None'}")
print("=" * 60)


CONFIGURATION
Mode: FILTERED KG
Input directory: /content/drive/MyDrive/Knowledge Graph
Output directory: /content/drive/MyDrive/kg_output_filtered
  Year minimum: 1970
  Top billing limit: 5
Dry-run limit rows: None
Dry-run limit movies: None


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory ready: {OUTPUT_DIR}")


Mounted at /content/drive
Output directory ready: /content/drive/MyDrive/kg_output_filtered


In [3]:
# Helper functions

def safe_split(value, sep=','):
    """Safely split a value, handling missing values marked as \\N"""
    if pd.isna(value) or value == '\\N' or value == '':
        return []
    return [v.strip() for v in str(value).split(sep) if v.strip() and v.strip() != '\\N']

def read_tsv_chunked(filepath, limit=None, chunksize=100000):
    """Read TSV file with optional row limit and chunking"""
    if limit:
        # For dry-run, read only first N rows
        df = pd.read_csv(filepath, sep='\t', nrows=limit, low_memory=False)
        return [df]  # Return as single chunk
    else:
        # Full processing with chunking
        return pd.read_csv(filepath, sep='\t', chunksize=chunksize, low_memory=False)

def get_file_path(filename):
    """Get full path for a file in INPUT_DIR"""
    return os.path.join(INPUT_DIR, filename)

print("Helper functions defined")


Helper functions defined


In [4]:
# Step 1: Extract entities and build triples from title.basics.tsv
# Entities: movies (tconst), genres
# Relations: (movie) --HAS_GENRE--> (genre)
# Apply filtering if USE_FILTERED_MODE is True

print("=" * 60)
print("Step 1: Processing title.basics.tsv")
print("=" * 60)

triples = []
movies = set()
genres = set()
kept_movies = set()  # Set of tconst that passed all filters (used in later steps)

basics_path = get_file_path(FILES['basics'])
print(f"Reading: {basics_path}")
if USE_FILTERED_MODE:
    print(f"Filtering: titleType='movie', startYear>={YEAR_MIN}, isAdult=0, genres not missing")

chunks = read_tsv_chunked(basics_path, limit=LIMIT_ROWS)
movies_processed = 0
movies_filtered_out = 0

for chunk_idx, chunk in enumerate(chunks):
    print(f"Processing chunk {chunk_idx + 1}... (rows: {len(chunk)})")

    # Filter out rows with missing tconst
    chunk = chunk[chunk['tconst'].notna() & (chunk['tconst'] != '\\N')]

    if USE_FILTERED_MODE:
        # Apply filtering rules
        # 1. Keep only titleType == "movie"
        chunk = chunk[chunk['titleType'] == 'movie']

        # 2. Exclude isAdult == 1
        chunk = chunk[(chunk['isAdult'].isna()) | (chunk['isAdult'] == 0) | (chunk['isAdult'] == '0')]

        # 3. Filter by startYear >= YEAR_MIN and not missing
        chunk = chunk[chunk['startYear'].notna() & (chunk['startYear'] != '\\N')]
        # Convert startYear to numeric, handling errors
        chunk['startYear_numeric'] = pd.to_numeric(chunk['startYear'], errors='coerce')
        chunk = chunk[chunk['startYear_numeric'] >= YEAR_MIN]

        # 4. Drop rows with missing genres
        chunk = chunk[chunk['genres'].notna() & (chunk['genres'] != '\\N') & (chunk['genres'] != '')]

    movies_processed += len(chunk)

    for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Chunk {chunk_idx + 1}"):
        movie_id = str(row['tconst']).strip()
        if not movie_id or movie_id == '\\N':
            continue

        # Check LIMIT_MOVIES if set
        if LIMIT_MOVIES and len(kept_movies) >= LIMIT_MOVIES:
            break

        movies.add(movie_id)
        kept_movies.add(movie_id)

        # Extract genres
        genre_list = safe_split(row.get('genres', ''))
        for genre in genre_list:
            if genre:
                genres.add(genre)
                triples.append(('HAS_GENRE', movie_id, genre))

    if LIMIT_MOVIES and len(kept_movies) >= LIMIT_MOVIES:
        print(f"Reached LIMIT_MOVIES={LIMIT_MOVIES}, stopping processing")
        break

movies_filtered_out = movies_processed - len(kept_movies)

print(f"\nStep 1 Complete:")
print(f"  Movies processed: {movies_processed:,}")
if USE_FILTERED_MODE:
    print(f"  Movies kept after filtering: {len(kept_movies):,}")
    print(f"  Movies filtered out: {movies_filtered_out:,}")
else:
    print(f"  Movies found: {len(movies):,}")
print(f"  Genres found: {len(genres)}")
print(f"  HAS_GENRE triples: {sum(1 for t in triples if t[0] == 'HAS_GENRE'):,}")


Step 1: Processing title.basics.tsv
Reading: /content/drive/MyDrive/Knowledge Graph/title.basics.tsv
Filtering: titleType='movie', startYear>=1970, isAdult=0, genres not missing
Processing chunk 1... (rows: 100000)


Chunk 1: 100%|██████████| 22597/22597 [00:01<00:00, 20714.08it/s]


Processing chunk 2... (rows: 100000)


Chunk 2: 100%|██████████| 25049/25049 [00:01<00:00, 22535.56it/s]


Processing chunk 3... (rows: 100000)


...

Processing chunk 119... (rows: 100000)


Chunk 119: 100%|██████████| 2151/2151 [00:00<00:00, 22251.03it/s]


Processing chunk 120... (rows: 55435)


Chunk 120: 100%|██████████| 1199/1199 [00:00<00:00, 22184.94it/s]



Step 1 Complete:
  Movies processed: 444,051
  Movies kept after filtering: 444,051
  Movies filtered out: 0
  Genres found: 26
  HAS_GENRE triples: 689,724


In [5]:
# Step 2: Extract relations from title.crew.tsv
# Relations: (person) --DIRECTED--> (movie), (person) --WROTE--> (movie)
# If filtered mode: only process movies in kept_movies set

print("=" * 60)
print("Step 2: Processing title.crew.tsv")
print("=" * 60)

persons = set()
crew_path = get_file_path(FILES['crew'])
print(f"Reading: {crew_path}")
if USE_FILTERED_MODE:
    print(f"Filtering: only movies in kept set ({len(kept_movies):,} movies)")

chunks = read_tsv_chunked(crew_path, limit=LIMIT_ROWS)
directed_count = 0
wrote_count = 0
crew_rows_processed = 0
crew_rows_filtered = 0

for chunk_idx, chunk in enumerate(chunks):
    print(f"Processing chunk {chunk_idx + 1}... (rows: {len(chunk)})")

    # Filter out rows with missing tconst
    chunk = chunk[chunk['tconst'].notna() & (chunk['tconst'] != '\\N')]
    crew_rows_processed += len(chunk)

    if USE_FILTERED_MODE:
        # Filter to only keep movies that passed the movie filter
        chunk['tconst_str'] = chunk['tconst'].astype(str).str.strip()
        chunk = chunk[chunk['tconst_str'].isin(kept_movies)]
        crew_rows_filtered += len(chunk)

    for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Chunk {chunk_idx + 1}"):
        movie_id = str(row['tconst']).strip()
        if not movie_id or movie_id == '\\N':
            continue

        # Double-check if in kept_movies (for filtered mode)
        if USE_FILTERED_MODE and movie_id not in kept_movies:
            continue

        # Extract directors
        directors = safe_split(row.get('directors', ''))
        for director_id in directors:
            if director_id:
                persons.add(director_id)
                triples.append(('DIRECTED', director_id, movie_id))
                directed_count += 1

        # Extract writers
        writers = safe_split(row.get('writers', ''))
        for writer_id in writers:
            if writer_id:
                persons.add(writer_id)
                triples.append(('WROTE', writer_id, movie_id))
                wrote_count += 1

print(f"\nStep 2 Complete:")
if USE_FILTERED_MODE:
    print(f"  Crew rows processed: {crew_rows_processed:,}")
    print(f"  Crew rows kept: {crew_rows_filtered:,}")
    print(f"  Crew rows filtered out: {crew_rows_processed - crew_rows_filtered:,}")
print(f"  Persons found so far: {len(persons):,}")
print(f"  DIRECTED triples: {directed_count:,}")
print(f"  WROTE triples: {wrote_count:,}")


Step 2: Processing title.crew.tsv
Reading: /content/drive/MyDrive/Knowledge Graph/title.crew.tsv
Filtering: only movies in kept set (444,051 movies)
Processing chunk 1... (rows: 100000)


Chunk 1: 100%|██████████| 22597/22597 [00:01<00:00, 18726.11it/s]


Processing chunk 2... (rows: 100000)


Chunk 2: 100%|██████████| 25049/25049 [00:01<00:00, 19758.07it/s]


Processing chunk 3... (rows: 100000)


...

Processing chunk 119... (rows: 100000)


Chunk 119: 100%|██████████| 2151/2151 [00:00<00:00, 21905.53it/s]


Processing chunk 120... (rows: 55435)


Chunk 120: 100%|██████████| 1199/1199 [00:00<00:00, 19406.91it/s]


Step 2 Complete:
  Crew rows processed: 11,955,435
  Crew rows kept: 444,051
  Crew rows filtered out: 11,511,384
  Persons found so far: 373,183
  DIRECTED triples: 493,393
  WROTE triples: 586,518


In [6]:
# Step 3: Extract relations from title.principals.tsv
# Relations: (person) --ACTED_IN--> (movie) where category in {"actor", "actress"}
# If filtered mode: only process movies in kept_movies set AND ordering <= TOP_BILLING

print("=" * 60)
print("Step 3: Processing title.principals.tsv")
print("=" * 60)

principals_path = get_file_path(FILES['principals'])
print(f"Reading: {principals_path}")
if USE_FILTERED_MODE:
    print(f"Filtering: only movies in kept set, ordering <= {TOP_BILLING}")

chunks = read_tsv_chunked(principals_path, limit=LIMIT_ROWS)
acted_count = 0
principals_rows_processed = 0
principals_rows_filtered = 0

# Valid acting categories
ACTING_CATEGORIES = {'actor', 'actress'}

for chunk_idx, chunk in enumerate(chunks):
    print(f"Processing chunk {chunk_idx + 1}... (rows: {len(chunk)})")

    # Filter for acting roles only
    chunk = chunk[
        chunk['tconst'].notna() &
        (chunk['tconst'] != '\\N') &
        chunk['nconst'].notna() &
        (chunk['nconst'] != '\\N') &
        chunk['category'].notna()
    ]

    # Filter by category
    chunk = chunk[chunk['category'].str.lower().isin(ACTING_CATEGORIES)]
    principals_rows_processed += len(chunk)

    if USE_FILTERED_MODE:
        # Filter to only keep movies that passed the movie filter
        chunk['tconst_str'] = chunk['tconst'].astype(str).str.strip()
        chunk = chunk[chunk['tconst_str'].isin(kept_movies)]

        # Filter by ordering <= TOP_BILLING
        # Convert ordering to numeric, handling missing values
        chunk['ordering_numeric'] = pd.to_numeric(chunk['ordering'], errors='coerce')
        chunk = chunk[chunk['ordering_numeric'].notna() & (chunk['ordering_numeric'] <= TOP_BILLING)]
        principals_rows_filtered += len(chunk)

    for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Chunk {chunk_idx + 1}"):
        movie_id = str(row['tconst']).strip()
        person_id = str(row['nconst']).strip()

        if movie_id and person_id and movie_id != '\\N' and person_id != '\\N':
            # Double-check if in kept_movies (for filtered mode)
            if USE_FILTERED_MODE and movie_id not in kept_movies:
                continue

            # Double-check ordering for filtered mode
            if USE_FILTERED_MODE:
                try:
                    ordering_val = pd.to_numeric(row.get('ordering'), errors='coerce')
                    if pd.isna(ordering_val) or ordering_val > TOP_BILLING:
                        continue
                except:
                    continue

            persons.add(person_id)
            triples.append(('ACTED_IN', person_id, movie_id))
            acted_count += 1

print(f"\nStep 3 Complete:")
if USE_FILTERED_MODE:
    print(f"  Principals rows processed: {principals_rows_processed:,}")
    print(f"  Principals rows kept: {principals_rows_filtered:,}")
    print(f"  Principals rows filtered out: {principals_rows_processed - principals_rows_filtered:,}")
print(f"  Total persons: {len(persons):,}")
print(f"  ACTED_IN triples: {acted_count:,}")


Step 3: Processing title.principals.tsv
Reading: /content/drive/MyDrive/Knowledge Graph/title.principals.tsv
Filtering: only movies in kept set, ordering <= 5
Processing chunk 1... (rows: 100000)


Chunk 1: 0it [00:00, ?it/s]


Processing chunk 2... (rows: 100000)


Chunk 2: 100%|██████████| 10/10 [00:00<00:00, 8750.90it/s]


Processing chunk 3... (rows: 100000)


...

Processing chunk 951... (rows: 100000)


Chunk 951: 100%|██████████| 806/806 [00:00<00:00, 19511.43it/s]


Processing chunk 952... (rows: 12527)


Chunk 952: 100%|██████████| 122/122 [00:00<00:00, 15433.72it/s]


Step 3 Complete:
  Principals rows processed: 39,624,377
  Principals rows kept: 1,493,177
  Principals rows filtered out: 38,131,200
  Total persons: 957,242
  ACTED_IN triples: 1,493,177


In [7]:
# Step 4: Create entity and relation mappings
# Map string entities/relations to integer IDs

print("=" * 60)
print("Step 4: Creating entity and relation mappings")
print("=" * 60)

# Collect all entities
all_entities = set()
all_entities.update(movies)
all_entities.update(persons)
all_entities.update(genres)

# Create entity mapping (string -> integer ID)
entity_map = {entity: idx for idx, entity in enumerate(sorted(all_entities), start=1)}
# Reserve 0 for padding if needed (optional)

# Create relation mapping
relations = {'HAS_GENRE', 'DIRECTED', 'WROTE', 'ACTED_IN'}
relation_map = {rel: idx for idx, rel in enumerate(sorted(relations), start=1)}

print(f"Entity mapping created: {len(entity_map)} entities")
print(f"Relation mapping created: {len(relation_map)} relations")
print(f"\nRelations: {sorted(relations)}")


Step 4: Creating entity and relation mappings
Entity mapping created: 1401319 entities
Relation mapping created: 4 relations

Relations: ['ACTED_IN', 'DIRECTED', 'HAS_GENRE', 'WROTE']


In [8]:
# Step 5: Convert triples to integer IDs and create final dataset

print("=" * 60)
print("Step 5: Converting triples to integer IDs")
print("=" * 60)

# Filter triples to only include entities that exist in our entity map
# (should be all, but safety check)
valid_triples = []
missing_entities = set()

for rel, head, tail in tqdm(triples, desc="Converting triples"):
    if head in entity_map and tail in entity_map and rel in relation_map:
        valid_triples.append({
            'head': entity_map[head],
            'relation': relation_map[rel],
            'tail': entity_map[tail],
            'head_str': head,
            'relation_str': rel,
            'tail_str': tail
        })
    else:
        if head not in entity_map:
            missing_entities.add(head)
        if tail not in entity_map:
            missing_entities.add(tail)

if missing_entities:
    print(f"Warning: {len(missing_entities)} entities not found in mapping (should not happen)")

print(f"Valid triples: {len(valid_triples)}")
print(f"Original triples: {len(triples)}")
print(f"Filtered out: {len(triples) - len(valid_triples)}")


Step 5: Converting triples to integer IDs


Converting triples: 100%|██████████| 3262812/3262812 [00:05<00:00, 633795.17it/s]

Valid triples: 3262812
Original triples: 3262812
Filtered out: 0


In [9]:
# Step 6: Save outputs to Google Drive

print("=" * 60)
print("Step 6: Saving outputs")
print("=" * 60)

# Determine output filenames based on mode
if USE_FILTERED_MODE:
    triples_filename = 'triples_filtered.csv'
    entity_map_filename = 'entity_map_filtered.csv'
    relation_map_filename = 'relation_map_filtered.csv'
else:
    triples_filename = 'triples.csv'
    entity_map_filename = 'entity_map.csv'
    relation_map_filename = 'relation_map.csv'

# Save triples.csv (with integer IDs)
triples_df = pd.DataFrame(valid_triples)
triples_df[['head', 'relation', 'tail']].to_csv(
    os.path.join(OUTPUT_DIR, triples_filename),
    index=False
)
print(f"Saved: {OUTPUT_DIR}/{triples_filename}")
print(f"  Shape: {triples_df.shape}")

# Save entity_map.csv
entity_map_df = pd.DataFrame([
    {'entity_id': idx, 'entity': entity}
    for entity, idx in sorted(entity_map.items(), key=lambda x: x[1])
])
entity_map_df.to_csv(
    os.path.join(OUTPUT_DIR, entity_map_filename),
    index=False
)
print(f"Saved: {OUTPUT_DIR}/{entity_map_filename}")
print(f"  Shape: {entity_map_df.shape}")

# Save relation_map.csv
relation_map_df = pd.DataFrame([
    {'relation_id': idx, 'relation': rel}
    for rel, idx in sorted(relation_map.items(), key=lambda x: x[1])
])
relation_map_df.to_csv(
    os.path.join(OUTPUT_DIR, relation_map_filename),
    index=False
)
print(f"Saved: {OUTPUT_DIR}/{relation_map_filename}")
print(f"  Shape: {relation_map_df.shape}")

print("\nAll outputs saved successfully!")


Step 6: Saving outputs
Saved: /content/drive/MyDrive/kg_output_filtered/triples_filtered.csv
  Shape: (3262812, 6)
Saved: /content/drive/MyDrive/kg_output_filtered/entity_map_filtered.csv
  Shape: (1401319, 2)
Saved: /content/drive/MyDrive/kg_output_filtered/relation_map_filtered.csv
  Shape: (4, 2)

All outputs saved successfully!


In [10]:
# Step 7: Print statistics

print("=" * 60)
print("KNOWLEDGE GRAPH STATISTICS")
print("=" * 60)

print(f"\nMode: {'FILTERED KG' if USE_FILTERED_MODE else 'FULL KG'}")

print(f"\nEntities:")
print(f"  Movies: {len(movies):,}")
print(f"  Persons: {len(persons):,}")
print(f"  Genres: {len(genres)}")
print(f"  Total entities: {len(all_entities):,}")

print(f"\nRelations:")
for rel in sorted(relations):
    count = sum(1 for t in valid_triples if t['relation_str'] == rel)
    print(f"  {rel}: {count:,} triples")

print(f"\nTotal triples: {len(valid_triples):,}")

# Comparison with full KG if in filtered mode
if USE_FILTERED_MODE and FULL_KG_STATS.get('triples'):
    reduction_ratio = len(valid_triples) / FULL_KG_STATS['triples']
    print(f"\nReduction vs Full KG:")
    print(f"  Full KG triples: {FULL_KG_STATS['triples']:,}")
    print(f"  Filtered KG triples: {len(valid_triples):,}")
    print(f"  Reduction ratio: {reduction_ratio:.2%}")
    print(f"  Size reduction: {(1 - reduction_ratio):.2%}")

if USE_FILTERED_MODE and FULL_KG_STATS.get('movies'):
    movie_reduction = len(movies) / FULL_KG_STATS['movies']
    print(f"\n  Full KG movies: {FULL_KG_STATS['movies']:,}")
    print(f"  Filtered KG movies: {len(movies):,}")
    print(f"  Movie reduction ratio: {movie_reduction:.2%}")

print(f"\nOutput files:")
if USE_FILTERED_MODE:
    print(f"  {OUTPUT_DIR}/triples_filtered.csv")
    print(f"  {OUTPUT_DIR}/entity_map_filtered.csv")
    print(f"  {OUTPUT_DIR}/relation_map_filtered.csv")
else:
    print(f"  {OUTPUT_DIR}/triples.csv")
    print(f"  {OUTPUT_DIR}/entity_map.csv")
    print(f"  {OUTPUT_DIR}/relation_map.csv")

print("\n" + "=" * 60)
print("Pipeline complete!")
print("=" * 60)


KNOWLEDGE GRAPH STATISTICS

Mode: FILTERED KG

Entities:
  Movies: 444,051
  Persons: 957,242
  Genres: 26
  Total entities: 1,401,319

Relations:
  ACTED_IN: 1,493,177 triples
  DIRECTED: 493,393 triples
  HAS_GENRE: 689,724 triples
  WROTE: 586,518 triples

Total triples: 3,262,812

Reduction vs Full KG:
  Full KG triples: 81,000,000
  Filtered KG triples: 3,262,812
  Reduction ratio: 4.03%
  Size reduction: 95.97%

  Full KG movies: 16,500,000
  Filtered KG movies: 444,051
  Movie reduction ratio: 2.69%

Output files:
  /content/drive/MyDrive/kg_output_filtered/triples_filtered.csv
  /content/drive/MyDrive/kg_output_filtered/entity_map_filtered.csv
  /content/drive/MyDrive/kg_output_filtered/relation_map_filtered.csv

Pipeline complete!


# TransE Training Stage with PyKEEN

This section trains TransE embeddings on the filtered Knowledge Graph.

**Steps:**
1. Load filtered triples from Google Drive
2. Create PyKEEN TriplesFactory
3. Train/validation/test split
4. Train TransE model
5. Evaluate with ranking metrics
6. Save embeddings and metadata

In [11]:
# Install PyKEEN and dependencies
%pip install pykeen -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.3 MB/s eta 0:00:00


In [12]:
# TransE Training Configuration
import pandas as pd
import numpy as np
import random
import json
import torch
from pathlib import Path
from pykeen.triples import TriplesFactory
from pykeen.evaluation import RankBasedEvaluator
from pykeen.pipeline import pipeline
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

# ============================================================================
# TRANSE TRAINING CONFIGURATION
# ============================================================================
EMBEDDING_DIM = 128
NUM_EPOCHS = 30
BATCH_SIZE = 512  # Reasonable for Colab GPU
LEARNING_RATE = 0.001
MARGIN = 1.0  # Margin for MarginRankingLoss
NEGATIVE_SAMPLER_KWARGS = {'num_negs_per_pos': 1}  # Default negative sampling

# Train/validation/test split ratios
TRAIN_RATIO = 0.8
VALID_RATIO = 0.1
TEST_RATIO = 0.1

# Paths
FILTERED_KG_DIR = '/content/drive/MyDrive/kg_output_filtered'
TRIPLES_FILE = f'{FILTERED_KG_DIR}/triples_filtered.csv'
ENTITY_MAP_FILE = f'{FILTERED_KG_DIR}/entity_map_filtered.csv'
RELATION_MAP_FILE = f'{FILTERED_KG_DIR}/relation_map_filtered.csv'
OUTPUT_DIR = f'{FILTERED_KG_DIR}/pykeen_transe'

# Dry-run mode: limit triples for quick testing
DRY_RUN_LIMIT = None  # Set to integer (e.g., 10000) for quick testing

# Device configuration
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print("=" * 60)
print("TRANSE TRAINING CONFIGURATION")
print("=" * 60)
print(f"Embedding dimension: {EMBEDDING_DIM}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Margin: {MARGIN}")
print(f"Device: {DEVICE}")
print(f"Train/Valid/Test split: {TRAIN_RATIO:.0%}/{VALID_RATIO:.0%}/{TEST_RATIO:.0%}")
print(f"Dry-run limit: {DRY_RUN_LIMIT if DRY_RUN_LIMIT else 'None (full dataset)'}")
print("=" * 60)

INFO:pykeen.utils:Using opt_einsum


TRANSE TRAINING CONFIGURATION
Embedding dimension: 128
Epochs: 30
Batch size: 512
Learning rate: 0.001
Margin: 1.0
Device: cuda
Train/Valid/Test split: 80%/10%/10%
Dry-run limit: None (full dataset)


In [13]:
# Mount Google Drive (if not already mounted)
from google.colab import drive
try:
    drive.mount('/content/drive', force_remount=False)
except:
    print("Drive already mounted or not in Colab environment")

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
print(f"Output directory ready: {OUTPUT_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Output directory ready: /content/drive/MyDrive/kg_output_filtered/pykeen_transe


In [14]:
# Helper function: Load triples from CSV
def load_triples(filepath, limit=None):
    """
    Load triples from CSV file.

    Args:
        filepath: Path to triples CSV file
        limit: Optional limit on number of rows to read (for dry-run)

    Returns:
        DataFrame with columns: head, relation, tail
    """
    print(f"Loading triples from: {filepath}")

    if limit:
        print(f"  Dry-run mode: limiting to {limit:,} rows")
        triples_df = pd.read_csv(filepath, nrows=limit)
    else:
        triples_df = pd.read_csv(filepath)

    print(f"  Loaded {len(triples_df):,} triples")
    print(f"  Columns: {list(triples_df.columns)}")

    # Ensure we have the expected columns
    required_cols = ['head', 'relation', 'tail']
    if not all(col in triples_df.columns for col in required_cols):
        raise ValueError(f"Expected columns {required_cols}, got {list(triples_df.columns)}")

    # Remove any rows with missing values
    initial_count = len(triples_df)
    triples_df = triples_df.dropna(subset=required_cols)
    if len(triples_df) < initial_count:
        print(f"  Removed {initial_count - len(triples_df):,} rows with missing values")

    # Convert to string and strip whitespace
    for col in required_cols:
        triples_df[col] = triples_df[col].astype(str).str.strip()

    # Remove empty strings
    triples_df = triples_df[
        (triples_df['head'] != '') &
        (triples_df['relation'] != '') &
        (triples_df['tail'] != '')
    ]

    print(f"  Final triples count: {len(triples_df):,}")
    return triples_df

# Load triples
triples_df = load_triples(TRIPLES_FILE, limit=DRY_RUN_LIMIT)

Loading triples from: /content/drive/MyDrive/kg_output_filtered/triples_filtered.csv
  Loaded 3,262,812 triples
  Columns: ['head', 'relation', 'tail']
  Final triples count: 3,262,812


In [15]:
# Helper function: Build TriplesFactory from labeled triples
def build_factory(triples_df, create_inverse_triples=False):
    """
    Create PyKEEN TriplesFactory from DataFrame with string labels.

    Args:
        triples_df: DataFrame with columns head, relation, tail (string labels)
        create_inverse_triples: Whether to create inverse triples

    Returns:
        TriplesFactory object
    """
    print("=" * 60)
    print("Building TriplesFactory")
    print("=" * 60)

    # Convert to numpy array of string triples
    triples_array = triples_df[['head', 'relation', 'tail']].values

    print(f"  Input triples shape: {triples_array.shape}")
    print(f"  Unique entities: {len(set(triples_array[:, 0]) | set(triples_array[:, 2])):,}")
    print(f"  Unique relations: {len(set(triples_array[:, 1]))}")

    # Create TriplesFactory from labeled triples
    factory = TriplesFactory.from_labeled_triples(
        triples=triples_array,
        create_inverse_triples=create_inverse_triples
    )

    print(f"  Factory created successfully")
    print(f"  Number of entities: {factory.num_entities:,}")
    print(f"  Number of relations: {factory.num_relations}")
    print(f"  Number of triples: {factory.num_triples:,}")

    return factory

# Build TriplesFactory
triples_factory = build_factory(triples_df, create_inverse_triples=False)

Building TriplesFactory
  Input triples shape: (3262812, 3)
  Unique entities: 1,401,319
  Unique relations: 4
  Factory created successfully
  Number of entities: 1,401,319
  Number of relations: 4
  Number of triples: 3,238,794


In [16]:
# Create train/validation/test split
print("=" * 60)
print("Creating train/validation/test split")
print("=" * 60)

training, validation, testing = triples_factory.split(
    ratios=[TRAIN_RATIO, VALID_RATIO, TEST_RATIO],
    random_state=RANDOM_SEED
)

print(f"Training triples: {training.num_triples:,} ({training.num_triples/triples_factory.num_triples:.1%})")
print(f"Validation triples: {validation.num_triples:,} ({validation.num_triples/triples_factory.num_triples:.1%})")
print(f"Test triples: {testing.num_triples:,} ({testing.num_triples/triples_factory.num_triples:.1%})")
print(f"Random seed: {RANDOM_SEED}")

Creating train/validation/test split


INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [1189743, 323879, 323880]


Training triples: 2,591,035 (80.0%)
Validation triples: 323,879 (10.0%)
Test triples: 323,880 (10.0%)
Random seed: 42


In [17]:
# Helper function: Train TransE model
def train_model(training, validation, testing, device='cuda'):
    """
    Train TransE model using PyKEEN pipeline.

    Args:
        training: Training TriplesFactory
        validation: Validation TriplesFactory
        testing: Testing TriplesFactory
        device: Device to use ('cuda' or 'cpu')

    Returns:
        Trained model and training results
    """
    print("=" * 60)
    print("Training TransE Model")
    print("=" * 60)
    print(f"Device: {device}")
    print(f"Embedding dimension: {EMBEDDING_DIM}")
    print(f"Epochs: {NUM_EPOCHS}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Learning rate: {LEARNING_RATE}")
    print("=" * 60)

    # Use PyKEEN pipeline for training
    # PyKEEN requires both training and testing to be specified
    result = pipeline(
        training=training,
        validation=validation,
        testing=testing,
        model='TransE',
        model_kwargs=dict(
            embedding_dim=EMBEDDING_DIM,
        ),
        training_kwargs=dict(
            num_epochs=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            use_tqdm=True,
        ),
        training_loop='SLCWA',
        negative_sampler='basic',
        negative_sampler_kwargs=NEGATIVE_SAMPLER_KWARGS,
        loss='marginranking',
        loss_kwargs=dict(margin=MARGIN),
        optimizer='Adam',
        optimizer_kwargs={'lr': LEARNING_RATE},
        device=device,
        random_seed=RANDOM_SEED,
    )

    print("\nTraining completed!")
    return result

# Train the model
training_result = train_model(training, validation, testing, device=DEVICE)
model = training_result.model

INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()


Training TransE Model
Device: cuda
Embedding dimension: 128
Epochs: 30
Batch size: 512
Learning rate: 0.001


INFO:pykeen.nn.representation:Inferred unique=False for Embedding()


Training epochs on cuda:0:   0%|          | 0/30 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/5.06k [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/5.06k [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/5.06k [00:00<?, ?batch/s]

Evaluating on cuda:0:   0%|          | 0.00/324k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 6796.25s seconds



Training completed!


In [18]:
# Fast evaluation: evaluate on subset of test triples for faster runtime
FAST_EVAL = True  # Set to True to evaluate on random subset (20,000 triples)
FAST_EVAL_SIZE = 20000  # Number of triples to use for fast evaluation

# Helper function: Evaluate model with ranking metrics
def evaluate_model(model, testing, training, validation, device='cuda'):
    """
    Evaluate model on test set using ranking metrics.

    Args:
        model: Trained PyKEEN model
        testing: Test TriplesFactory
        training: Training TriplesFactory (for filtering)
        validation: Validation TriplesFactory (for filtering)
        device: Device to use

    Returns:
        Dictionary of evaluation metrics
    """
    print("=" * 60)
    print("Evaluating Model on Test Set")
    print("=" * 60)

    evaluator = RankBasedEvaluator(metrics=["hits@k", "mrr", "mr"])

    # Move model to device
    model = model.to(device)

    # Prepare test triples
    test_triples = testing.mapped_triples

    # Fast evaluation: use random subset if enabled
    if FAST_EVAL and len(test_triples) > FAST_EVAL_SIZE:
        print(f"Fast evaluation enabled: sampling {FAST_EVAL_SIZE:,} triples from {len(test_triples):,} test triples")
        # Set random seed for reproducibility
        generator = torch.Generator(device=device)
        generator.manual_seed(RANDOM_SEED)

        # Randomly sample indices
        indices = torch.randperm(len(test_triples))[:FAST_EVAL_SIZE]
        test_triples = test_triples[indices]
        print(f"  Using {len(test_triples):,} triples for evaluation")
    else:
        print(f"Evaluating on full test set: {len(test_triples):,} triples")

    # Prepare additional filter triples (training and validation sets)
    # These are used to filter out known triples during evaluation
    additional_filter_triples = [
        training.mapped_triples,
        validation.mapped_triples
    ]

    # Evaluate
    metrics = evaluator.evaluate(
        model=model,
        mapped_triples=test_triples,
        additional_filter_triples=additional_filter_triples,
        batch_size=BATCH_SIZE,
        device=device,
    )

    # Convert RankBasedMetricResults to dictionary
    metrics_dict = metrics.to_dict()

    print("\nRanking Metrics:")
    print(f"  Hits@1: {metrics_dict.get('hits_at_1', 0):.4f}")
    print(f"  Hits@3: {metrics_dict.get('hits_at_3', 0):.4f}")
    print(f"  Hits@10: {metrics_dict.get('hits_at_10', 0):.4f}")
    print(f"  Mean Rank: {metrics_dict.get('mean_rank', 0):.2f}")
    print(f"  Mean Reciprocal Rank: {metrics_dict.get('mean_reciprocal_rank', 0):.4f}")

    return metrics_dict

# Evaluate model
test_metrics = evaluate_model(model, testing, training, validation, device=DEVICE)

Evaluating Model on Test Set
Fast evaluation enabled: sampling 20,000 triples from 323,880 test triples
  Using 20,000 triples for evaluation


Evaluating on cuda:0:   0%|          | 0.00/20.0k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 427.21s seconds



Ranking Metrics:
  Hits@1: 0.0000
  Hits@3: 0.0000
  Hits@10: 0.0000
  Mean Rank: 0.00
  Mean Reciprocal Rank: 0.0000


In [20]:
m = test_metrics["both"]["realistic"]

print("Hits@1:", m["hits_at_1"])
print("Hits@3:", m["hits_at_3"])
print("Hits@10:", m["hits_at_10"])

# نزدیک‌ترین معادل MR/MRR در این خروجی‌ها:
print("Mean Rank (arithmetic):", m["arithmetic_mean_rank"])
print("Harmonic Mean Rank:", m["harmonic_mean_rank"])
print("Inverse Harmonic Mean Rank:", m["inverse_harmonic_mean_rank"])


Hits@1: 0.0005
Hits@3: 0.061375
Hits@10: 0.087825
Mean Rank (arithmetic): 171739.734375
Harmonic Mean Rank: 27.99739646911621
Inverse Harmonic Mean Rank: 0.03571760654449463


In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

5692

In [21]:
# Helper function: Compute Precision@K, Recall@K, F1@K
def compute_precision_recall_f1(model, testing, k=10, device='cuda', batch_size=512):
    """
    Compute Precision@K, Recall@K, and F1@K for test triples.

    Args:
        model: Trained PyKEEN model
        testing: Test TriplesFactory
        k: Top-K for precision/recall
        device: Device to use
        batch_size: Batch size for evaluation

    Returns:
        Dictionary with precision@k, recall@k, f1@k
    """
    print(f"\nComputing Precision@{k}, Recall@{k}, F1@{k}...")

    model = model.to(device)
    model.eval()

    test_triples = testing.mapped_triples.to(device)
    num_triples = len(test_triples)

    true_positives = 0
    total_positive = num_triples

    # Process in batches
    with torch.no_grad():
        for i in range(0, num_triples, batch_size):
            batch = test_triples[i:i+batch_size]

            # For each triple, get top-k predictions for tail
            h, r, t = batch[:, 0], batch[:, 1], batch[:, 2]

            # Get scores for all possible tails
            scores = model.score_t(batch)

            # Get top-k predictions
            _, top_k_indices = torch.topk(scores, k=k, dim=1)

            # Check if true tail is in top-k
            true_tails = t.unsqueeze(1)
            matches = (top_k_indices == true_tails).any(dim=1)
            true_positives += matches.sum().item()

    precision_k = true_positives / (num_triples * k) if num_triples > 0 else 0.0
    recall_k = true_positives / total_positive if total_positive > 0 else 0.0
    f1_k = 2 * (precision_k * recall_k) / (precision_k + recall_k) if (precision_k + recall_k) > 0 else 0.0

    return {
        f'precision_at_{k}': precision_k,
        f'recall_at_{k}': recall_k,
        f'f1_at_{k}': f1_k
    }

# Compute Precision@10, Recall@10, F1@10
prf_metrics = compute_precision_recall_f1(model, testing, k=10, device=DEVICE, batch_size=BATCH_SIZE)
print(f"  Precision@10: {prf_metrics['precision_at_10']:.4f}")
print(f"  Recall@10: {prf_metrics['recall_at_10']:.4f}")
print(f"  F1@10: {prf_metrics['f1_at_10']:.4f}")

# Merge with test_metrics
test_metrics.update(prf_metrics)


Computing Precision@10, Recall@10, F1@10...


OutOfMemoryError: CUDA out of memory. Tried to allocate 342.12 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.59 GiB is free. Process 2932 has 12.15 GiB memory in use. Of the allocated memory 2.21 GiB is allocated by PyTorch, and 9.83 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Step 1: MovieLens to IMDb Linkage

This section links MovieLens 20M movies to IMDb titles (movieId -> tconst).

**Goal:** Create a high-coverage mapping using title/year matching.

**Approach:**
1. Parse MovieLens titles to extract year and normalize titles
2. Build an index from IMDb title.basics.tsv (filtered for movies)
3. Match using normalized title + year, with fallback strategies
4. Output linkage CSV and summary statistics

In [26]:
# MovieLens-IMDb Linkage Configuration
import pandas as pd
import numpy as np
import json
import re
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# ============================================================================
# LINKAGE CONFIGURATION
# ============================================================================
DRIVE_ROOT = '/content/drive/MyDrive'

# Input paths
ML_MOVIES_FILE = f'{DRIVE_ROOT}/Knowledge Graph/movies.csv'  # MovieLens movies file
IMDB_BASICS_FILE = f'{DRIVE_ROOT}/Knowledge Graph/title.basics.tsv'  # IMDb title.basics.tsv

# Output path
OUTPUT_DIR = f'{DRIVE_ROOT}/ml_imdb_linkage'

# Processing configuration
IMDB_CHUNKSIZE = 100000  # Chunk size for reading IMDb TSV

print("=" * 60)
print("MOVIELENS-IMDB LINKAGE CONFIGURATION")
print("=" * 60)
print(f"MovieLens movies file: {ML_MOVIES_FILE}")
print(f"IMDb basics file: {IMDB_BASICS_FILE}")
print(f"Output directory: {OUTPUT_DIR}")
print("=" * 60)

MOVIELENS-IMDB LINKAGE CONFIGURATION
MovieLens movies file: /content/drive/MyDrive/Knowledge Graph/movies.csv
IMDb basics file: /content/drive/MyDrive/Knowledge Graph/title.basics.tsv
Output directory: /content/drive/MyDrive/ml_imdb_linkage


In [27]:
# Mount Google Drive (if not already mounted)
from google.colab import drive
try:
    drive.mount('/content/drive', force_remount=False)
except:
    print("Drive already mounted or not in Colab environment")

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
print(f"Output directory ready: {OUTPUT_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Output directory ready: /content/drive/MyDrive/ml_imdb_linkage


In [36]:
# Helper function: Process MovieLens title (handle articles and alternate titles)
def process_ml_title(ml_title):
    """
    Process MovieLens title to handle:
    - Trailing articles: "X, The" -> "The X"
    - Alternate titles: remove "(a.k.a. ...)" or "(...)" before extracting year
    - Extract main title and year

    Args:
        ml_title: MovieLens title string

    Returns:
        Tuple of (main_title, year_int_or_None, original_title)
    """
    import unicodedata

    if pd.isna(ml_title) or ml_title == '':
        return '', None, str(ml_title) if not pd.isna(ml_title) else ''

    original_title = str(ml_title).strip()
    title_str = original_title

    # Step 1: Handle trailing articles before processing parentheses
    # Pattern: "X, The" -> "The X", "X, A" -> "A X", "X, An" -> "An X"
    article_patterns = [
        (r'^(.+?),\s+The\s*$', r'The \1'),
        (r'^(.+?),\s+A\s*$', r'A \1'),
        (r'^(.+?),\s+An\s*$', r'An \1')
    ]

    for pattern, replacement in article_patterns:
        if re.match(pattern, title_str, re.IGNORECASE):
            title_str = re.sub(pattern, replacement, title_str, flags=re.IGNORECASE).strip()
            break

    # Step 2: Remove alternate titles (everything from first "(" that's not the year)
    # Keep only the main title part before any parentheses
    # But we need to preserve the final "(YYYY)" if it exists
    # Strategy: extract year first, then strip everything from first "("

    # Extract year from last "(YYYY)" at end
    year = None
    year_match = re.search(r'\((\d{4})\)\s*$', title_str)
    if year_match:
        year = int(year_match.group(1))
        # Temporarily remove year for processing
        title_str_no_year = re.sub(r'\s*\(\d{4}\)\s*$', '', title_str).strip()
    else:
        title_str_no_year = title_str

    # Remove everything from first "(" onward (alternate titles, a.k.a., etc.)
    # Example: "Arena, The (a.k.a. Naked Warriors)" -> "Arena, The"
    # But we already moved "The" to front, so: "The Arena (a.k.a. Naked Warriors)" -> "The Arena"
    main_title = title_str_no_year
    first_paren_idx = main_title.find('(')
    if first_paren_idx >= 0:
        main_title = main_title[:first_paren_idx].strip()

    # If we removed the year parentheses, add it back conceptually
    # (year is already extracted above)

    return main_title, year, original_title

# Helper function: Extract title and year from MovieLens title
def extract_title_year(ml_title):
    """
    Extract title and year from MovieLens title format.
    Uses process_ml_title() to handle articles and alternate titles.

    Args:
        ml_title: MovieLens title string

    Returns:
        Tuple of (title_without_year, year_int_or_None)
    """
    main_title, year, _ = process_ml_title(ml_title)
    return main_title, year

# Helper function: Robust title normalization with unicode normalization
def normalize_title(text):
    """
    Robust title normalization for matching:
    - Unicode normalization (remove accents)
    - Lowercase
    - Strip leading/trailing whitespace
    - Convert "&" to "and"
    - Remove punctuation/symbols (keep alphanumerics and spaces only)
    - Collapse multiple spaces into one
    - Remove common stop words at beginning: "the", "a", "an"

    Args:
        text: Title string

    Returns:
        Normalized title string
    """
    import unicodedata

    if pd.isna(text) or text == '':
        return ''

    # Unicode normalization: remove accents (NFD + remove combining marks)
    # Convert to unicode string and normalize
    text_str = str(text)
    # Normalize to NFD (decomposed), then remove combining characters
    text_str = unicodedata.normalize('NFD', text_str)
    text_str = ''.join(c for c in text_str if unicodedata.category(c) != 'Mn')

    # Convert to string and lowercase
    title_str = text_str.lower().strip()

    # Convert "&" to "and"
    title_str = title_str.replace('&', 'and')

    # Remove punctuation/symbols (keep alphanumeric and spaces only)
    title_str = re.sub(r'[^\w\s]', ' ', title_str)

    # Collapse multiple spaces into one
    title_str = re.sub(r'\s+', ' ', title_str)

    # Strip leading/trailing spaces
    title_str = title_str.strip()

    # Remove common stop words at beginning only
    stop_words = ['the', 'a', 'an']
    words = title_str.split()
    if words and words[0] in stop_words:
        title_str = ' '.join(words[1:]).strip()

    return title_str

# Test normalization and extraction
test_titles = [
    "Toy Story (1995)",
    "The Shawshank Redemption (1994)",
    "Forrest Gump",
    "Pulp Fiction (1994)",
    "The Matrix (1999)",
    "Men in Black (1997)",
    "Men & Women (2000)",
    "Arena, The (a.k.a. Naked Warriors) (1974)",
    "Brute (Bandyta) (1998)",
    "Shakespeare, The (1996)",
    "Café Lumière (2003)"  # Test unicode normalization
]
print("Title extraction and normalization examples:")
for title in test_titles:
    title_clean, year = extract_title_year(title)
    norm = normalize_title(title_clean)
    print(f"  '{title}' -> clean: '{title_clean}', year: {year}, normalized: '{norm}'")

Title extraction and normalization examples:
  'Toy Story (1995)' -> clean: 'Toy Story', year: 1995, normalized: 'toy story'
  'The Shawshank Redemption (1994)' -> clean: 'The Shawshank Redemption', year: 1994, normalized: 'shawshank redemption'
  'Forrest Gump' -> clean: 'Forrest Gump', year: None, normalized: 'forrest gump'
  'Pulp Fiction (1994)' -> clean: 'Pulp Fiction', year: 1994, normalized: 'pulp fiction'
  'The Matrix (1999)' -> clean: 'The Matrix', year: 1999, normalized: 'matrix'
  'Men in Black (1997)' -> clean: 'Men in Black', year: 1997, normalized: 'men in black'
  'Men & Women (2000)' -> clean: 'Men & Women', year: 2000, normalized: 'men and women'
  'Arena, The (a.k.a. Naked Warriors) (1974)' -> clean: 'Arena, The', year: 1974, normalized: 'arena the'
  'Brute (Bandyta) (1998)' -> clean: 'Brute', year: 1998, normalized: 'brute'
  'Shakespeare, The (1996)' -> clean: 'Shakespeare, The', year: 1996, normalized: 'shakespeare the'
  'Café Lumière (2003)' -> clean: 'Café Lum

In [37]:
# Helper function: Load MovieLens movies
def load_movielens_movies(filepath):
    """
    Load MovieLens movies file and extract year/normalize titles.
    Handles articles and alternate titles using process_ml_title().

    Args:
        filepath: Path to movies.csv

    Returns:
        DataFrame with columns: movieId, title, genres, ml_title_clean, ml_title_norm, ml_year
    """
    print(f"Loading MovieLens movies from: {filepath}")

    df = pd.read_csv(filepath)
    print(f"  Loaded {len(df):,} movies")

    # Extract title and year using helper function (handles articles and alternates)
    title_year_pairs = df['title'].apply(extract_title_year)
    df['ml_title_clean'] = [pair[0] for pair in title_year_pairs]
    df['ml_year'] = [pair[1] for pair in title_year_pairs]
    df['ml_year'] = df['ml_year'].astype('Int64')  # Int64 allows NaN

    # Normalize cleaned titles (without year)
    df['ml_title_norm'] = df['ml_title_clean'].apply(normalize_title)

    # Count statistics
    movies_with_year = df['ml_year'].notna().sum()
    print(f"  Movies with year: {movies_with_year:,} ({movies_with_year/len(df)*100:.1f}%)")
    print(f"  Movies without year: {(df['ml_year'].isna().sum()):,}")
    print(f"  Sample processed titles:")
    for idx, row in df.head(5).iterrows():
        print(f"    '{row['title']}' -> clean: '{row['ml_title_clean']}' -> norm: '{row['ml_title_norm']}' (year: {row['ml_year']})")

    return df

# Load MovieLens movies
ml_movies = load_movielens_movies(ML_MOVIES_FILE)
print(f"\nSample MovieLens movies:")
print(ml_movies[['movieId', 'title', 'ml_year', 'ml_title_norm']].head(10))

Loading MovieLens movies from: /content/drive/MyDrive/Knowledge Graph/movies.csv
  Loaded 27,278 movies
  Movies with year: 27,255 (99.9%)
  Movies without year: 23
  Sample processed titles:
    'Toy Story (1995)' -> clean: 'Toy Story' -> norm: 'toy story' (year: 1995)
    'Jumanji (1995)' -> clean: 'Jumanji' -> norm: 'jumanji' (year: 1995)
    'Grumpier Old Men (1995)' -> clean: 'Grumpier Old Men' -> norm: 'grumpier old men' (year: 1995)
    'Waiting to Exhale (1995)' -> clean: 'Waiting to Exhale' -> norm: 'waiting to exhale' (year: 1995)
    'Father of the Bride Part II (1995)' -> clean: 'Father of the Bride Part II' -> norm: 'father of the bride part ii' (year: 1995)

Sample MovieLens movies:
   movieId                               title  ml_year  \
0        1                    Toy Story (1995)     1995   
1        2                      Jumanji (1995)     1995   
2        3             Grumpier Old Men (1995)     1995   
3        4            Waiting to Exhale (1995)     1995   

In [38]:
# Helper function: Stream IMDb basics and build index
def build_imdb_index_streaming(filepath, chunksize=100000):
    """
    Stream IMDb title.basics.tsv and build index for fast matching.

    Filter criteria:
    - titleType == "movie"
    - isAdult == 0

    Index structure:
    {
        normalized_title: [
            {'tconst': 'tt...', 'year': int or None, 'imdb_title_used': 'original title'},
            ...
        ]
    }

    Important: Include BOTH primaryTitle and originalTitle, avoid duplicates.

    Args:
        filepath: Path to title.basics.tsv
        chunksize: Chunk size for reading TSV

    Returns:
        Dictionary mapping normalized_title -> list of candidates
    """
    print("=" * 60)
    print("Building IMDb Index")
    print("=" * 60)
    print(f"Reading: {filepath}")
    print(f"Chunk size: {chunksize:,} rows")

    index = defaultdict(list)
    seen_tconst_title = set()  # Track (tconst, normalized_title) to avoid duplicates
    total_rows = 0
    filtered_rows = 0

    # Read in chunks
    chunks = pd.read_csv(filepath, sep='\t', chunksize=chunksize, low_memory=False)

    for chunk_idx, chunk in enumerate(chunks):
        total_rows += len(chunk)

        # Filter early: titleType == "movie" and isAdult == 0
        chunk = chunk[
            (chunk['titleType'] == 'movie') &
            (chunk['isAdult'].isin([0, '0']) | chunk['isAdult'].isna())
        ]
        filtered_rows += len(chunk)

        # Process each row
        for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc=f"Chunk {chunk_idx + 1}", leave=False):
            tconst = str(row['tconst']).strip()
            if not tconst or tconst == '\\N':
                continue

            # Extract year (handle "\N" as missing)
            year = None
            start_year = row.get('startYear')
            if pd.notna(start_year) and str(start_year) != '\\N':
                try:
                    year = int(float(str(start_year)))
                except:
                    pass

            # Process primaryTitle
            if pd.notna(row.get('primaryTitle')):
                primary_title = str(row['primaryTitle']).strip()
                if primary_title:
                    title_norm = normalize_title(primary_title)
                    if title_norm:
                        key = (tconst, title_norm)
                        if key not in seen_tconst_title:
                            seen_tconst_title.add(key)
                            index[title_norm].append({
                                'tconst': tconst,
                                'year': year,
                                'imdb_title_used': primary_title
                            })

            # Process originalTitle (if different from primaryTitle)
            if pd.notna(row.get('originalTitle')):
                original_title = str(row['originalTitle']).strip()
                if original_title:
                    orig_title_norm = normalize_title(original_title)
                    if orig_title_norm:
                        key = (tconst, orig_title_norm)
                        if key not in seen_tconst_title:
                            seen_tconst_title.add(key)
                            index[orig_title_norm].append({
                                'tconst': tconst,
                                'year': year,
                                'imdb_title_used': original_title
                            })

        if (chunk_idx + 1) % 10 == 0:
            print(f"  Processed {chunk_idx + 1} chunks, {total_rows:,} total rows, {filtered_rows:,} movies")

    print(f"\nIndex built:")
    print(f"  Total rows processed: {total_rows:,}")
    print(f"  Movies filtered: {filtered_rows:,}")
    print(f"  Unique normalized titles in index: {len(index):,}")
    print(f"  Total index entries: {sum(len(v) for v in index.values()):,}")

    return dict(index)

# Build IMDb index
imdb_index = build_imdb_index_streaming(IMDB_BASICS_FILE, chunksize=IMDB_CHUNKSIZE)

Building IMDb Index
Reading: /content/drive/MyDrive/Knowledge Graph/title.basics.tsv
Chunk size: 100,000 rows


  Processed 10 chunks, 1,000,000 total rows, 220,833 movies


  Processed 20 chunks, 2,000,000 total rows, 261,783 movies


  Processed 30 chunks, 3,000,000 total rows, 299,779 movies


  Processed 40 chunks, 4,000,000 total rows, 339,379 movies


...

  Processed 80 chunks, 8,000,000 total rows, 517,916 movies


  Processed 90 chunks, 9,000,000 total rows, 568,267 movies


  Processed 100 chunks, 10,000,000 total rows, 627,273 movies


  Processed 110 chunks, 11,000,000 total rows, 680,031 movies


  Processed 120 chunks, 11,955,435 total rows, 718,637 movies

Index built:
  Total rows processed: 11,955,435
  Movies filtered: 718,637
  Unique normalized titles in index: 682,749
  Total index entries: 808,356


In [39]:
# Helper function: Relaxed title match (fallback)
def relaxed_match(norm_title, imdb_index):
    """
    Attempt relaxed matching strategies:
    1. Remove spaces from normalized title
    2. Remove subtitle after ":" and retry

    Simple deterministic approach without fuzzy matching.

    Args:
        norm_title: Normalized title
        imdb_index: IMDb index dictionary

    Returns:
        List of candidates or None
    """
    # Strategy 1: Remove all spaces
    title_no_spaces = norm_title.replace(' ', '')
    if title_no_spaces:
        candidates = []
        for key, values in imdb_index.items():
            key_no_spaces = key.replace(' ', '')
            if key_no_spaces == title_no_spaces:
                candidates.extend(values)
        if candidates:
            return candidates

    # Strategy 2: Remove subtitle after ":"
    if ':' in norm_title:
        title_no_subtitle = norm_title.split(':', 1)[0].strip()
        if title_no_subtitle:
            candidates = imdb_index.get(title_no_subtitle)
            if candidates:
                return candidates

    return None

# Helper function: Match MovieLens movies to IMDb
def match_movielens_to_imdb(ml_movies, imdb_index):
    """
    Match MovieLens movies to IMDb titles using two-stage strategy.

    Stage A: Title-only match
    - Lookup candidates by normalized_title
    - If exactly 1 candidate: match_type="title_only_unique"
    - If multiple candidates:
       - If ML year available: exact year match -> "title_year_exact"
       - Else: closest year (diff <= 1) -> "title_year_closest"
       - Else: ambiguous

    Stage B: Fallback (if still unmatched)
    - Attempt relaxed match (remove spaces)

    Args:
        ml_movies: DataFrame with MovieLens movies
        imdb_index: Dictionary mapping normalized_title -> candidates

    Returns:
        DataFrame with linkage results, match_type_counts, unmatched_samples, ambiguous_samples
    """
    print("=" * 60)
    print("Matching Movies")
    print("=" * 60)

    results = []
    match_type_counts = defaultdict(int)
    unmatched_samples = []
    ambiguous_samples = []

    # Track indices for random sampling
    unmatched_indices = []
    ambiguous_indices = []

    for idx, ml_row in tqdm(ml_movies.iterrows(), total=len(ml_movies), desc="Matching"):
        movie_id = ml_row['movieId']
        ml_title_raw = ml_row['title']
        ml_title_norm = ml_row['ml_title_norm']
        ml_year = ml_row['ml_year']

        # Initialize result
        result = {
            'movieId': movie_id,
            'ml_title_raw': ml_title_raw,
            'ml_title_norm': ml_title_norm,
            'ml_year': ml_year if pd.notna(ml_year) else None,
            'tconst': None,
            'imdb_title': None,
            'imdb_year': None,
            'match_type': 'unmatched'
        }

        # Skip if normalized title is empty
        if not ml_title_norm:
            results.append(result)
            match_type_counts['unmatched'] += 1
            unmatched_indices.append(idx)
            continue

        # STAGE A: Title-only match
        candidates = imdb_index.get(ml_title_norm, [])

        if not candidates:
            # No candidates found - try Stage B fallback
            relaxed_candidates = relaxed_match(ml_title_norm, imdb_index)
            if relaxed_candidates:
                candidates = relaxed_candidates
                # Use relaxed match logic
                if len(candidates) == 1:
                    match = candidates[0]
                    result['tconst'] = match['tconst']
                    result['imdb_title'] = match['imdb_title_used']
                    result['imdb_year'] = match['year']
                    result['match_type'] = 'title_only_unique'  # Relaxed match
                    results.append(result)
                    match_type_counts['title_only_unique'] += 1
                    continue
                # For multiple relaxed candidates, continue to main logic below
            else:
                # Still no candidates
                results.append(result)
                match_type_counts['unmatched'] += 1
                unmatched_indices.append(idx)
                continue

        # Process candidates
        if len(candidates) == 1:
            # Single candidate - match
            match = candidates[0]
            result['tconst'] = match['tconst']
            result['imdb_title'] = match['imdb_title_used']
            result['imdb_year'] = match['year']
            result['match_type'] = 'title_only_unique'
            results.append(result)
            match_type_counts['title_only_unique'] += 1
            continue

        # Multiple candidates - use year to disambiguate
        if pd.notna(ml_year):
            # Try exact year match first
            year_matches = [c for c in candidates if c['year'] == ml_year]
            if year_matches:
                match = year_matches[0]  # Take first if multiple
                result['tconst'] = match['tconst']
                result['imdb_title'] = match['imdb_title_used']
                result['imdb_year'] = match['year']
                result['match_type'] = 'title_year_exact'
                results.append(result)
                match_type_counts['title_year_exact'] += 1
                continue

            # Try closest year (only if difference <= 1)
            candidates_with_year = [c for c in candidates if c['year'] is not None]
            if candidates_with_year:
                best_match = min(candidates_with_year, key=lambda c: abs(c['year'] - ml_year))
                year_diff = abs(best_match['year'] - ml_year)
                if year_diff <= 1:
                    result['tconst'] = best_match['tconst']
                    result['imdb_title'] = best_match['imdb_title_used']
                    result['imdb_year'] = best_match['year']
                    result['match_type'] = 'title_year_closest'
                    results.append(result)
                    match_type_counts['title_year_closest'] += 1
                    continue

        # Multiple candidates but can't disambiguate
        result['match_type'] = 'ambiguous'
        results.append(result)
        match_type_counts['ambiguous'] += 1
        ambiguous_indices.append(idx)

    # Create DataFrame
    linkage_df = pd.DataFrame(results)

    # Print statistics
    print(f"\nMatching complete:")
    print(f"  Total movies: {len(linkage_df):,}")
    print(f"  Match type distribution:")
    for match_type, count in sorted(match_type_counts.items(), key=lambda x: -x[1]):
        pct = count / len(linkage_df) * 100
        print(f"    {match_type}: {count:,} ({pct:.1f}%)")

    matched_count = len(linkage_df[linkage_df['tconst'].notna()])
    coverage_pct = matched_count / len(linkage_df) * 100
    print(f"\n  Matched movies: {matched_count:,}")
    print(f"  Coverage: {coverage_pct:.2f}%")

    # Collect random samples for debugging (20 unmatched, 20 ambiguous)
    np.random.seed(RANDOM_SEED)
    if len(unmatched_indices) > 0:
        sample_unmatched_idx = np.random.choice(unmatched_indices, size=min(20, len(unmatched_indices)), replace=False)
        for idx in sample_unmatched_idx:
            ml_row = ml_movies.loc[idx]
            unmatched_samples.append((
                ml_row['movieId'],
                ml_row['title'],
                ml_row['ml_title_clean'],
                ml_row['ml_year']
            ))

    if len(ambiguous_indices) > 0:
        sample_ambiguous_idx = np.random.choice(ambiguous_indices, size=min(20, len(ambiguous_indices)), replace=False)
        for idx in sample_ambiguous_idx:
            ml_row = ml_movies.loc[idx]
            norm_title = ml_row['ml_title_norm']
            candidates = imdb_index.get(norm_title, [])
            candidate_years = [c['year'] for c in candidates if c['year'] is not None]
            ambiguous_samples.append((
                ml_row['movieId'],
                ml_row['title'],
                ml_row['ml_title_clean'],
                ml_row['ml_year'],
                candidate_years
            ))

    return linkage_df, match_type_counts, unmatched_samples, ambiguous_samples

# Match movies
linkage_df, match_type_counts, unmatched_samples, ambiguous_samples = match_movielens_to_imdb(ml_movies, imdb_index)

Matching Movies


Matching: 100%|██████████| 27278/27278 [25:38<00:00, 17.73it/s]


Matching complete:
  Total movies: 27,278
  Match type distribution:
    title_only_unique: 11,622 (42.6%)
    unmatched: 8,041 (29.5%)
    title_year_exact: 6,836 (25.1%)
    ambiguous: 484 (1.8%)
    title_year_closest: 295 (1.1%)

  Matched movies: 18,753
  Coverage: 68.75%


In [40]:
# Print debugging information
print("=" * 60)
print("DEBUGGING: Random Unmatched Samples")
print("=" * 60)
print(f"Showing {len(unmatched_samples)} random unmatched titles:")
for i, (movie_id, raw_title, processed_title, ml_year) in enumerate(unmatched_samples, 1):
    print(f"  {i}. movieId: {movie_id}, Raw: '{raw_title}'")
    print(f"      Processed: '{processed_title}', Year: {ml_year}")

print("\n" + "=" * 60)
print("DEBUGGING: Random Ambiguous Samples")
print("=" * 60)
print(f"Showing {len(ambiguous_samples)} random ambiguous cases:")
for i, (movie_id, raw_title, processed_title, ml_year, candidate_years) in enumerate(ambiguous_samples, 1):
    print(f"  {i}. movieId: {movie_id}, Raw: '{raw_title}'")
    print(f"      Processed: '{processed_title}', ML Year: {ml_year}, Candidate Years: {candidate_years}")

# Helper function: Save outputs
def save_outputs(linkage_df, match_type_counts, output_dir):
    """
    Save linkage results to CSV and summary to JSON.

    Args:
        linkage_df: DataFrame with linkage results
        match_type_counts: Dictionary with match type counts
        output_dir: Output directory path
    """
    print("\n" + "=" * 60)
    print("Saving Outputs")
    print("=" * 60)

    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    # Save linkage CSV
    linkage_file = output_path / 'movielens_imdb_linkage.csv'
    linkage_df.to_csv(linkage_file, index=False)
    print(f"Saved: {linkage_file}")
    print(f"  Shape: {linkage_df.shape}")

    # Calculate summary statistics
    total_movies = len(linkage_df)
    matched_movies = len(linkage_df[linkage_df['tconst'].notna()])
    coverage_pct = matched_movies / total_movies * 100 if total_movies > 0 else 0.0

    # Create summary dictionary
    summary = {
        'total_movies': int(total_movies),
        'matched_movies': int(matched_movies),
        'coverage_pct': float(coverage_pct),
        'counts_by_match_type': {k: int(v) for k, v in match_type_counts.items()}
    }

    # Save summary JSON
    summary_file = output_path / 'linkage_summary.json'
    with open(summary_file, 'w') as f:
        json.dump(summary, f, indent=2)
    print(f"Saved: {summary_file}")

    print("\nSummary:")
    print(f"  Total movies: {summary['total_movies']:,}")
    print(f"  Matched movies: {summary['matched_movies']:,}")
    print(f"  Coverage: {summary['coverage_pct']:.2f}%")

    return summary

# Save outputs
summary = save_outputs(linkage_df, match_type_counts, OUTPUT_DIR)

DEBUGGING: Random Unmatched Samples
Showing 20 random unmatched titles:
  1. movieId: 41025, Raw: 'Good Woman, A (2004)'
      Processed: 'Good Woman, A', Year: 2004
  2. movieId: 3005, Raw: 'Bone Collector, The (1999)'
      Processed: 'Bone Collector, The', Year: 1999
  3. movieId: 56945, Raw: 'Perfect Holiday, The (2007)'
      Processed: 'Perfect Holiday, The', Year: 2007
  4. movieId: 116030, Raw: 'Children of the Corn 666: Isaac's Return (1999)'
      Processed: 'Children of the Corn 666: Isaac's Return', Year: 1999
  5. movieId: 5565, Raw: 'Dogwalker, The (2002)'
      Processed: 'Dogwalker, The', Year: 2002
  6. movieId: 96430, Raw: 'Odd Life of Timothy Green, The (2012)'
      Processed: 'Odd Life of Timothy Green, The', Year: 2012
  7. movieId: 95756, Raw: 'Cat in the Hat, The (1971)'
      Processed: 'Cat in the Hat, The', Year: 1971
  8. movieId: 1997, Raw: 'Exorcist, The (1973)'
      Processed: 'Exorcist, The', Year: 1973
  9. movieId: 121879, Raw: 'Method to the Madness 

In [41]:
# Print final summary and sample matches
print("=" * 60)
print("MOVIELENS-IMDB LINKAGE SUMMARY")
print("=" * 60)

print(f"\nCoverage Statistics:")
print(f"  Total MovieLens movies: {summary['total_movies']:,}")
print(f"  Successfully matched: {summary['matched_movies']:,}")
print(f"  Coverage: {summary['coverage_pct']:.2f}%")

print(f"\nMatch Type Breakdown:")
for match_type, count in sorted(summary['counts_by_match_type'].items(), key=lambda x: -x[1]):
    pct = count / summary['total_movies'] * 100
    print(f"  {match_type}: {count:,} ({pct:.1f}%)")

print(f"\nOutput Files:")
print(f"  {OUTPUT_DIR}/movielens_imdb_linkage.csv")
print(f"  {OUTPUT_DIR}/linkage_summary.json")

print(f"\nSample Matches:")
sample_matched = linkage_df[linkage_df['tconst'].notna()].head(10)
print(sample_matched[['movieId', 'ml_title_raw', 'ml_year', 'tconst', 'imdb_title', 'imdb_year', 'match_type']].to_string(index=False))

print("\n" + "=" * 60)
print("Linkage Complete!")
print("=" * 60)

MOVIELENS-IMDB LINKAGE SUMMARY

Coverage Statistics:
  Total MovieLens movies: 27,278
  Successfully matched: 18,753
  Coverage: 68.75%

Match Type Breakdown:
  title_only_unique: 11,622 (42.6%)
  unmatched: 8,041 (29.5%)
  title_year_exact: 6,836 (25.1%)
  ambiguous: 484 (1.8%)
  title_year_closest: 295 (1.1%)

Output Files:
  /content/drive/MyDrive/ml_imdb_linkage/movielens_imdb_linkage.csv
  /content/drive/MyDrive/ml_imdb_linkage/linkage_summary.json

Sample Matches:
 movieId                       ml_title_raw  ml_year    tconst                  imdb_title  imdb_year        match_type
       1                   Toy Story (1995)   1995.0 tt0114709                   Toy Story     1995.0 title_only_unique
       2                     Jumanji (1995)   1995.0 tt0113497                     Jumanji     1995.0 title_only_unique
       3            Grumpier Old Men (1995)   1995.0 tt0113228            Grumpier Old Men     1995.0 title_only_unique
       4           Waiting to Exhale (1995)  

In [42]:
# Helper function: Save embeddings and metadata
def save_artifacts(model, triples_factory, training, validation, testing, test_metrics, output_dir):
    """
    Save model embeddings, mappings, and metadata to disk.

    Args:
        model: Trained PyKEEN model
        triples_factory: TriplesFactory used for training
        training_result: Training result object
        testing: Test TriplesFactory
        test_metrics: Test set evaluation metrics
        output_dir: Output directory path
    """
    print("=" * 60)
    print("Saving Artifacts")
    print("=" * 60)

    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    # Extract embeddings
    entity_embeddings = model.entity_representations[0](indices=None).detach().cpu().numpy()
    relation_embeddings = model.relation_representations[0](indices=None).detach().cpu().numpy()

    print(f"Entity embeddings shape: {entity_embeddings.shape}")
    print(f"Relation embeddings shape: {relation_embeddings.shape}")

    # Save embeddings as numpy arrays
    np.save(output_path / 'entity_embeddings.npy', entity_embeddings)
    print(f"  Saved: {output_path / 'entity_embeddings.npy'}")

    np.save(output_path / 'relation_embeddings.npy', relation_embeddings)
    print(f"  Saved: {output_path / 'relation_embeddings.npy'}")

    # Save entity and relation mappings
    entity_to_id = {entity: int(idx) for entity, idx in triples_factory.entity_to_id.items()}
    relation_to_id = {relation: int(idx) for relation, idx in triples_factory.relation_to_id.items()}

    with open(output_path / 'entity_to_id.json', 'w') as f:
        json.dump(entity_to_id, f, indent=2)
    print(f"  Saved: {output_path / 'entity_to_id.json'}")

    with open(output_path / 'relation_to_id.json', 'w') as f:
        json.dump(relation_to_id, f, indent=2)
    print(f"  Saved: {output_path / 'relation_to_id.json'}")

    # Create metadata dictionary
    metadata = {
        'model': 'TransE',
        'embedding_dim': EMBEDDING_DIM,
        'num_epochs': NUM_EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'margin': MARGIN,
        'random_seed': RANDOM_SEED,
        'device': DEVICE,
        'dataset_sizes': {
            'total_triples': int(triples_factory.num_triples),
            'train_triples': int(training.num_triples),
            'valid_triples': int(validation.num_triples),
            'test_triples': int(testing.num_triples),
        },
        'num_entities': int(triples_factory.num_entities),
        'num_relations': int(triples_factory.num_relations),
        'evaluation_metrics': {k: float(v) if isinstance(v, (int, float, np.number)) else str(v)
                               for k, v in test_metrics.items()},
        'training_loss': float(training_result.losses[-1]) if hasattr(training_result, 'losses') and training_result.losses else None,
    }

    # Save metadata
    with open(output_path / 'metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"  Saved: {output_path / 'metadata.json'}")

    # Try to save model checkpoint if available
    try:
        checkpoint_path = output_path / 'model_checkpoint.pt'
        torch.save({
            'model_state_dict': model.state_dict(),
            'model_kwargs': {'embedding_dim': EMBEDDING_DIM},
            'entity_to_id': entity_to_id,
            'relation_to_id': relation_to_id,
        }, checkpoint_path)
        print(f"  Saved: {checkpoint_path}")
    except Exception as e:
        print(f"  Warning: Could not save model checkpoint: {e}")

    print("\nAll artifacts saved successfully!")
    return metadata

# Save all artifacts
metadata = save_artifacts(model, triples_factory, training, validation, testing, test_metrics, OUTPUT_DIR)

Saving Artifacts
Entity embeddings shape: (1401319, 128)
Relation embeddings shape: (4, 128)
  Saved: /content/drive/MyDrive/ml_imdb_linkage/entity_embeddings.npy
  Saved: /content/drive/MyDrive/ml_imdb_linkage/relation_embeddings.npy
  Saved: /content/drive/MyDrive/ml_imdb_linkage/entity_to_id.json
  Saved: /content/drive/MyDrive/ml_imdb_linkage/relation_to_id.json
  Saved: /content/drive/MyDrive/ml_imdb_linkage/metadata.json
  Saved: /content/drive/MyDrive/ml_imdb_linkage/model_checkpoint.pt

All artifacts saved successfully!


In [43]:
# Print final summary
print("=" * 60)
print("TRANSE TRAINING SUMMARY")
print("=" * 60)

print(f"\nModel Configuration:")
print(f"  Model: TransE")
print(f"  Embedding dimension: {EMBEDDING_DIM}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")

print(f"\nDataset:")
print(f"  Total entities: {triples_factory.num_entities:,}")
print(f"  Total relations: {triples_factory.num_relations}")
print(f"  Total triples: {triples_factory.num_triples:,}")
print(f"  Train: {training.num_triples:,} ({training.num_triples/triples_factory.num_triples:.1%})")
print(f"  Valid: {validation.num_triples:,} ({validation.num_triples/triples_factory.num_triples:.1%})")
print(f"  Test: {testing.num_triples:,} ({testing.num_triples/triples_factory.num_triples:.1%})")

print(f"\nTest Set Evaluation:")
for metric_name, metric_value in test_metrics.items():
    if isinstance(metric_value, (int, float, np.number)):
        print(f"  {metric_name}: {metric_value:.4f}")

print(f"\nSaved Artifacts:")
print(f"  {OUTPUT_DIR}/entity_embeddings.npy")
print(f"  {OUTPUT_DIR}/relation_embeddings.npy")
print(f"  {OUTPUT_DIR}/entity_to_id.json")
print(f"  {OUTPUT_DIR}/relation_to_id.json")
print(f"  {OUTPUT_DIR}/metadata.json")

print("\n" + "=" * 60)
print("TransE Training Complete!")
print("=" * 60)

TRANSE TRAINING SUMMARY

Model Configuration:
  Model: TransE
  Embedding dimension: 128
  Epochs: 30
  Batch size: 512
  Learning rate: 0.001

Dataset:
  Total entities: 1,401,319
  Total relations: 4
  Total triples: 3,238,794
  Train: 2,591,035 (80.0%)
  Valid: 323,879 (10.0%)
  Test: 323,880 (10.0%)

Test Set Evaluation:

Saved Artifacts:
  /content/drive/MyDrive/ml_imdb_linkage/entity_embeddings.npy
  /content/drive/MyDrive/ml_imdb_linkage/relation_embeddings.npy
  /content/drive/MyDrive/ml_imdb_linkage/entity_to_id.json
  /content/drive/MyDrive/ml_imdb_linkage/relation_to_id.json
  /content/drive/MyDrive/ml_imdb_linkage/metadata.json

TransE Training Complete!


## movieId → kg_embedding